Universidad Torcuato Di Tella

Licenciatura en Tecnología Digital\
**Tecnología Digital VI: Inteligencia Artificial**


In [32]:
%pip install torchaudio
%pip install  pydub
%pip install soundfile
%pip install wandb
import wandb
import os
import torch
import torchaudio
import soundfile as sf
import tarfile
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
from torch.utils.data import Dataset
from torchaudio.datasets import GTZAN
from torch.utils.data import DataLoader
import torchaudio.transforms as tt
from torch.utils.data import random_split
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline


^C
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: typing-extensions<5,>=4.8 in c:\users\laras\miniconda3\envs\metodos\lib\site-packages (from wandb) (4.14.0)




# TP3: Encodeador de música



## Orden de pasos

0. Elijan GPU para que corra mas rapido (RAM --> change runtime type --> T4 GPU)
1. Descargamos el dataset y lo descomprimimos en alguna carpeta en nuestro drive.
2. Conectamos la notebook a gdrive y seteamos data_dir con el path a los archivos.
3. Visualización de los archivos
4. Clasificación
5. Evaluación




In [ ]:
project_name='Music_genre_classification'

batch_size = 128
learning_rate = 0.0005
epochs = 50

#Experimentos NN + waveform

# experiment_name = 'Iden_1capa_densa'
#experiment_name = 'Iden_3capas_NOdensa_1906_128_100'
#experiment_name = 'Iden_3capas_NOdensa_1906_128_dropout'
# experiment_name = 'Iden_3capas_densa_1906_128_dropout'
# experiment_name = 'Iden_6capas_densa_sin_dropout'
#experiment_name = 'Iden_6capas_NOdensa_sin_dropout'
# experiment_name = 'Iden_6capas_NOdensa_dropout'
# experiment_name = 'Iden_12capas_NOdensa_sindropout'
# experiment_name = 'Iden_12capas_NOdensa_dropout04'
# experiment_name = 'Iden_12capas_densa_dropout04'
# experiment_name = 'Iden_12capas_densa_dropout08'

# Experimentos CNN + espectrograma
# experiment_name = 'CNN_2conv_1capa'
experiment_name = 'CNN_2conv_5capas'



In [ ]:
# Start a new wandb run to track this script.
run = wandb.init(
    # Set the wandb entity where your project will be logged (generally your team name).
    entity="larabarijhoff-universidad-torcuato-di-tella",
    # Set the wandb project where this run will be logged.
    project="tp3-td6-25",
    # Track hyperparameters and run metadata.
    name = experiment_name
)


### Experimentos NN

In [ ]:
if experiment_name == 'Iden_1capa_densa':
    model = nn.Sequential(
    nn.Linear(110250, 10), #primera capa oculta
)
    
if experiment_name == 'Iden_3capas_NOdensa_1906_128_100':
    model = nn.Sequential(
    nn.Linear(110250, 50), #primera capa oculta
    nn.Identity(),
    nn.Linear(50, 40), #segunda capa oculta
    nn.Identity(),
    nn.Linear(40, 10)  # tercera capa
)

if experiment_name == 'Iden_3capas_NOdensa_1906_128_100':
    model = nn.Sequential(
    nn.Linear(110250, 50), #primera capa oculta
    nn.Identity(),
    nn.Linear(50, 40), #segunda capa oculta
    nn.Identity(),
    nn.Linear(40, 10)  # tercera capa
)

if experiment_name == 'Iden_3capas_NOdensa_1906_128_dropout':
    model = nn.Sequential(
    nn.Linear(110250, 50), #primera capa oculta
    nn.Identity(),
    nn.Dropout(0.8),
    nn.Linear(50, 40), #segunda capa oculta
    nn.Identity(),
    nn.Dropout(0.8),
    nn.Linear(40, 10)  # tercera capa
)

if experiment_name == 'Iden_3capas_densa_1906_128_dropout':
    model = nn.Sequential(
    nn.Linear(110250, 1024), #primera capa oculta
    nn.Identity(),
    nn.Dropout(0.8),
    nn.Linear(1024, 512), #segunda capa oculta
    nn.Identity(),
    nn.Dropout(0.8),
    nn.Linear(512, 10)  # tercera capa
)
    
if experiment_name == 'Iden_6capas_densa_sin_dropout':
    model = nn.Sequential(
        nn.Linear(110250, 1024),
        nn.Identity(),
        nn.Linear(1024, 512),
        nn.Identity(),
        nn.Linear(512, 256),
        nn.Identity(),
        nn.Linear(256, 128),
        nn.Identity(),
        nn.Linear(128, 64),
        nn.Identity(),
        nn.Linear(64, 10)
)
if experiment_name == 'Iden_6capas_NOdensa_sindropout':
    model = nn.Sequential(
        nn.Linear(110250, 50), nn.Identity(),
        nn.Linear(50, 45), nn.Identity(),
        nn.Linear(45, 40), nn.Identity(),
        nn.Linear(40, 35), nn.Identity(),
        nn.Linear(35, 30), nn.Identity(),
        nn.Linear(30, 10)
)
if experiment_name == 'Iden_6capas_NOdensa_dropout':
    model = nn.Sequential(
        nn.Linear(110250, 50), nn.Identity(), nn.Dropout(0.8),
        nn.Linear(50, 45), nn.Identity(), nn.Dropout(0.8),
        nn.Linear(45, 40), nn.Identity(), nn.Dropout(0.8),
        nn.Linear(40, 35), nn.Identity(), nn.Dropout(0.8),
        nn.Linear(35, 30), nn.Identity(), nn.Dropout(0.8),
        nn.Linear(30, 10)
    )

if experiment_name == 'Iden_12capas_NOdensa_sindropout':
    model = nn.Sequential(
        nn.Linear(110250, 60), nn.Identity(),
        nn.Linear(60, 55), nn.Identity(),
        nn.Linear(55, 50), nn.Identity(),
        nn.Linear(50, 45), nn.Identity(),
        nn.Linear(45, 40), nn.Identity(),
        nn.Linear(40, 35), nn.Identity(),
        nn.Linear(35, 30), nn.Identity(),
        nn.Linear(30, 25), nn.Identity(),
        nn.Linear(25, 20), nn.Identity(),
        nn.Linear(20, 15), nn.Identity(),
        nn.Linear(15, 12), nn.Identity(),
        nn.Linear(12, 10)
)

if experiment_name == 'Iden_12capas_NOdensa_dropout04':
    model = nn.Sequential(
        nn.Linear(110250, 60), nn.Identity(), nn.Dropout(0.4),
        nn.Linear(60, 55), nn.Identity(), nn.Dropout(0.4),
        nn.Linear(55, 50), nn.Identity(), nn.Dropout(0.4),
        nn.Linear(50, 45), nn.Identity(), nn.Dropout(0.4),
        nn.Linear(45, 40), nn.Identity(), nn.Dropout(0.4),
        nn.Linear(40, 35), nn.Identity(), nn.Dropout(0.4),
        nn.Linear(35, 30), nn.Identity(), nn.Dropout(0.4),
        nn.Linear(30, 25), nn.Identity(), nn.Dropout(0.4),
        nn.Linear(25, 20), nn.Identity(), nn.Dropout(0.4),
        nn.Linear(20, 15), nn.Identity(), nn.Dropout(0.4),
        nn.Linear(15, 12), nn.Identity(), nn.Dropout(0.4),
        nn.Linear(12, 10)
)
if experiment_name == 'Iden_12capas_densa_dropout04':
    model = nn.Sequential(
        nn.Linear(110250, 2048), nn.Identity(), nn.Dropout(0.4),
        nn.Linear(2048, 1024), nn.Identity(), nn.Dropout(0.4),
        nn.Linear(1024, 1024), nn.Identity(), nn.Dropout(0.4),
        nn.Linear(1024, 512), nn.Identity(), nn.Dropout(0.4),
        nn.Linear(512, 512), nn.Identity(), nn.Dropout(0.4),
        nn.Linear(512, 256), nn.Identity(), nn.Dropout(0.4),
        nn.Linear(256, 256), nn.Identity(), nn.Dropout(0.4),
        nn.Linear(256, 128), nn.Identity(), nn.Dropout(0.4),
        nn.Linear(128, 128), nn.Identity(), nn.Dropout(0.4),
        nn.Linear(128, 64), nn.Identity(), nn.Dropout(0.4),
        nn.Linear(64, 32), nn.Identity(), nn.Dropout(0.4),
        nn.Linear(32, 10)
)
if experiment_name == 'Iden_12capas_densa_dropout08':
    model = nn.Sequential(
        nn.Linear(110250, 2048), nn.Identity(), nn.Dropout(0.8),
        nn.Linear(2048, 1024), nn.Identity(), nn.Dropout(0.8),
        nn.Linear(1024, 1024), nn.Identity(), nn.Dropout(0.8),
        nn.Linear(1024, 512), nn.Identity(), nn.Dropout(0.8),
        nn.Linear(512, 512), nn.Identity(), nn.Dropout(0.8),
        nn.Linear(512, 256), nn.Identity(), nn.Dropout(0.8),
        nn.Linear(256, 256), nn.Identity(), nn.Dropout(0.8),
        nn.Linear(256, 128), nn.Identity(), nn.Dropout(0.8),
        nn.Linear(128, 128), nn.Identity(), nn.Dropout(0.8),
        nn.Linear(128, 64), nn.Identity(), nn.Dropout(0.8),
        nn.Linear(64, 32), nn.Identity(), nn.Dropout(0.8),
        nn.Linear(32, 10)
)



### Experimentos CNN

In [ ]:
if experiment_name == 'CNN_2conv_1capa':
    model = nn.Sequential(
    # Primera capa convolucional
    nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1),  # Conv2D
    nn.Identity(),                                                                 # Identity
    nn.MaxPool2d(kernel_size=2, stride=2),                                         # MaxPool2D

    # Segunda capa convolucional
    nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1), # Conv2D
    nn.Identity(),                                                                  # Identity
    nn.MaxPool2d(kernel_size=2, stride=2),                                          # MaxPool2D

    # Capa totalmente conectada para la clasificación
    nn.Flatten(),                                                                   # Aplanar la salida
    nn.Linear(int(64 * (201 // 4) * (552 // 4)), 10),                                 # Capa densa
    nn.Identity()                                                                   # Identity
)
    
if experiment_name == 'CNN_2conv_5capas':
    model = nn.Sequential(
        nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1),
        nn.MaxPool2d(kernel_size=2, stride=2),
        
        nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),
        nn.MaxPool2d(kernel_size=2, stride=2),

        nn.Flatten(),
        
        nn.Linear(64 * (201 // 4) * (552 // 4), 512),
        nn.Linear(512, 256),
        nn.Linear(256, 128),
        nn.Linear(128, 64),
        nn.Linear(64, 10)
)


### 2. Conectamos la notebook a gdrive y seteamos data_dir con el path a los archivos.





data_dir es el path donde pusimos la carpeta genres. "'//content/drive/MyDrive/Materias/TD6 - Inteligencia Artificial/TPs/2023/TP4/genres/'" es un ejemplo. Modificar.

In [ ]:
import os
data_dir="..\genres_5sec" #para lari
#data_dir="../genres_5sec" #para piki
list_files=os.listdir(data_dir)
classes=[]
for file in list_files:
  name='{}/{}'.format(data_dir,file)
  if os.path.isdir(name):
    classes.append(file)
classes

In [ ]:
samplerate=22050
def parse_genres(fname):
    parts = fname.split('/')[-1].split('.')[0]
    return parts #' '.join(parts[0])

class MusicDataset(Dataset):
    def __init__(self, root):
        super().__init__()
        self.root = root
        self.files =[]
        for c in classes:
          self.files = self.files + [fname for fname in os.listdir(os.path.join(root,c)) if fname.endswith('.wav')]
        self.classes = list(set(parse_genres(fname) for fname in self.files))
        #self.transform = transform

    def __len__(self):
        return len(self.files)

    def __getitem__(self, i):
        fname = self.files[i]

        #img = self.transform(open_image(fpath))
        genre = parse_genres(fname)
        fpath = os.path.join(self.root,genre, fname)
        class_idx = self.classes.index(genre)
        audio = torchaudio.load(fpath)[0]

        return audio, class_idx
dataset = MusicDataset(data_dir)

### 3. Visualización de los archivos

In [ ]:
# Proba cargar un archivo válido lara
waveform, sr = torchaudio.load(r'..\genres_5sec\blues\blues.00000.wav') #para lari
#waveform, sr = torchaudio.load(r'../genres_5sec/blues/blues.00000.wav') #para piki
print(waveform.shape, sr)

In [ ]:
waveform,label= dataset[0]
print("shape of waveform {}, sample rate with {}, label is {} ".format(waveform.size(),samplerate,label))

In [ ]:
specgram=tt.Spectrogram()(waveform)
print("shape of spectogram {}".format(specgram.size()))

plt.figure(figsize=(20, 5))
plt.imshow(specgram.log2()[0,:,:].numpy(),cmap='magma')


In [ ]:
print("Waveform: {}\n".format(waveform))

plt.figure()
plt.plot(waveform.t().numpy())

Escuchamos el espectograma con la librería de audio

In [ ]:
import IPython
IPython.display.Audio(waveform,rate=samplerate)

In [ ]:
specgram.size()

In [ ]:
# Fijar la semilla para que la división sea reproducible
random_seed = 42
torch.manual_seed(random_seed)

# Calcular tamaños basados en porcentajes
total_size = len(dataset)
train_size = int(0.8 * total_size)
val_size = int(0.1 * total_size)
test_size = total_size - train_size - val_size

# Dividir el dataset
train_ds, val_ds, test_ds = random_split(dataset, [train_size, val_size, test_size])

# Comprobación de los tamaños de cada subconjunto
len(train_ds), len(val_ds), len(test_ds)

In [ ]:
from torch.utils.data import DataLoader

train_dl = DataLoader(train_ds, batch_size, shuffle=True, num_workers=0, pin_memory=True)
# Prueba para verificar si el DataLoader de entrenamiento funciona correctamente
try:
    for data, labels in train_dl:
        print("Forma de los datos:", data.shape)
        print("Etiquetas:", labels)
        break  # Carga solo el primer batch
except Exception as e:
    print("Error al cargar los datos del train_dl:", e)
valid_dl = DataLoader(val_ds, batch_size*2, num_workers=0, pin_memory=True)
test_dl = DataLoader(test_ds,1, num_workers=0, pin_memory=True)

### 4. Clasificación

### NN

In [ ]:
# dada por ellos:

# #1 Capa densa
# class MLP(nn.Module):
#     def __init__(self, n_input, hidden_dim=64, n_output=10):
#         super(MLP, self).__init__()
#         # Define layers dynamically
#         self.fc1 = nn.Linear(n_input, n_output)  # First hidden layer

#     def forward(self, x):
#        x = self.fc1(x)
#        return x
    
#3 Capas NO Densas
#La de mejor resultado dentro de NN en cuanto a cantidad de capas y densidad
# class MLP(nn.Module):
#     def __init__(self, n_input, hidden_dim=64, n_output=10):
#         super(MLP, self).__init__()
#         # Define layers dynamically
#         self.fc1 = nn.Linear(n_input, 50)  # First hidden layer
#         self.fc2 = nn.Linear(50, 40)  # Second hidden layer
#         self.fc3 = nn.Linear(40, n_output)  # Third hidden layer

#     def forward(self, x):
#         x = self.fc1(x)
#         x = self.fc2(x)
#         x = self.fc3(x)
#         return x

#3 Capas NO Densas con Dropout
#La de mejor resultado dentro de NN en cuanto a cantidad de capas y densidad
# class MLP(nn.Module):
#     def __init__(self, n_input, hidden_dim=64, n_output=10):
#         super(MLP, self).__init__()
#         # Define layers dynamically
#         self.fc1 = nn.Linear(n_input, 50)  # First hidden layer
#         self.fc2 = nn.Linear(50, 40)  # Second hidden layer
#         self.fc3 = nn.Linear(40, n_output)  # Third hidden layer
#         self.dropout = nn.Dropout(p=0.8)  # Dropout layer with probability p

#     def forward(self, x):
#         x = self.fc1(x)
#         x = self.dropout(x)        # Dropout
#         x = self.fc2(x)
#         x = self.dropout(x)        # Dropout
#         x = self.fc3(x)
#         return x

# #3 Capas Densas con Dropout
# #La de mejor resultado dentro de NN en cuanto a cantidad de capas y densidad
# class MLP(nn.Module):
#     def __init__(self, n_input, hidden_dim=64, n_output=10):
#         super(MLP, self).__init__()
#         # Define layers dynamically
#         self.fc1 = nn.Linear(n_input, 2048)  # First hidden layer
#         self.fc2 = nn.Linear(2048, 1024)  # Second hidden layer
#         self.fc3 = nn.Linear(1024, n_output)  # Third hidden layer
#         self.dropout = nn.Dropout(p=0.8)  # Dropout layer with probability p

#     def forward(self, x):
#         x = self.fc1(x)
#         x = self.dropout(x)        # Dropout
#         x = self.fc2(x)
#         x = self.dropout(x)        # Dropout
#         x = self.fc3(x)
#         return x

# 6 Capas Densas sin Dropout
# class MLP(nn.Module):
#     def __init__(self, n_input, n_output=10):
#         super(MLP, self).__init__()
#         self.fc1 = nn.Linear(n_input, 1024)
#         self.fc2 = nn.Linear(1024, 512)
#         self.fc3 = nn.Linear(512, 256)
#         self.fc4 = nn.Linear(256, 128)
#         self.fc5 = nn.Linear(128, 64)
#         self.fc6 = nn.Linear(64, n_output)

#     def forward(self, x):
#         x = self.fc1(x)
#         x = self.fc2(x)
#         x = self.fc3(x)
#         x = self.fc4(x)
#         x = self.fc5(x)
#         x = self.fc6(x)
#         return x

# #6 Capas no Densas sin Dropout
# class MLP(nn.Module):
#     def __init__(self, n_input=110250, n_output=10):
#         super(MLP, self).__init__()
#         self.fc1 = nn.Linear(n_input, 50)
#         self.fc2 = nn.Linear(50, 45)
#         self.fc3 = nn.Linear(45, 40)
#         self.fc4 = nn.Linear(40, 35)
#         self.fc5 = nn.Linear(35, 30)
#         self.fc6 = nn.Linear(30, n_output)

#     def forward(self, x):
#         x = self.fc1(x)
#         x = self.fc2(x)
#         x = self.fc3(x)
#         x = self.fc4(x)
#         x = self.fc5(x)
#         x = self.fc6(x)
#         return x

# #6 Capas no Densas con Dropout
# class MLP(nn.Module):
#     def __init__(self, n_input=110250, n_output=10):
#         super(MLP, self).__init__()
#         self.fc1 = nn.Linear(n_input, 50)
#         self.fc2 = nn.Linear(50, 45)
#         self.fc3 = nn.Linear(45, 40)
#         self.fc4 = nn.Linear(40, 35)
#         self.fc5 = nn.Linear(35, 30)
#         self.fc6 = nn.Linear(30, n_output)
#         self.dropout = nn.Dropout(0.8)

#     def forward(self, x):
#         x = self.dropout(self.fc1(x))
#         x = self.dropout(self.fc2(x))
#         x = self.dropout(self.fc3(x))
#         x = self.dropout(self.fc4(x))
#         x = self.dropout(self.fc5(x))
#         x = self.fc6(x)
#         return x


# 12 Capas no Densas sin Dropout
# class MLP(nn.Module):
#     def __init__(self, n_input=110250, n_output=10):
#         super(MLP, self).__init__()
#         self.fc1 = nn.Linear(n_input, 60)
#         self.fc2 = nn.Linear(60, 55)
#         self.fc3 = nn.Linear(55, 50)
#         self.fc4 = nn.Linear(50, 45)
#         self.fc5 = nn.Linear(45, 40)
#         self.fc6 = nn.Linear(40, 35)
#         self.fc7 = nn.Linear(35, 30)
#         self.fc8 = nn.Linear(30, 25)
#         self.fc9 = nn.Linear(25, 20)
#         self.fc10 = nn.Linear(20, 15)
#         self.fc11 = nn.Linear(15, 12)
#         self.fc12 = nn.Linear(12, n_output)

#     def forward(self, x):
#         x = self.fc1(x)
#         x = self.fc2(x)
#         x = self.fc3(x)
#         x = self.fc4(x)
#         x = self.fc5(x)
#         x = self.fc6(x)
#         x = self.fc7(x)
#         x = self.fc8(x)
#         x = self.fc9(x)
#         x = self.fc10(x)
#         x = self.fc11(x)
#         x = self.fc12(x)
#         return x

# # 12 Capas no Densas con Dropout
# class MLP(nn.Module):
#     def __init__(self, n_input=110250, n_output=10):
#         super(MLP, self).__init__()
#         self.fc1 = nn.Linear(n_input, 60)
#         self.fc2 = nn.Linear(60, 55)
#         self.fc3 = nn.Linear(55, 50)
#         self.fc4 = nn.Linear(50, 45)
#         self.fc5 = nn.Linear(45, 40)
#         self.fc6 = nn.Linear(40, 35)
#         self.fc7 = nn.Linear(35, 30)
#         self.fc8 = nn.Linear(30, 25)
#         self.fc9 = nn.Linear(25, 20)
#         self.fc10 = nn.Linear(20, 15)
#         self.fc11 = nn.Linear(15, 12)
#         self.fc12 = nn.Linear(12, n_output)
#         self.dropout = nn.Dropout(0.4)

#     def forward(self, x):
#         x = self.dropout(self.fc1(x))
#         x = self.dropout(self.fc2(x))
#         x = self.dropout(self.fc3(x))
#         x = self.dropout(self.fc4(x))
#         x = self.dropout(self.fc5(x))
#         x = self.dropout(self.fc6(x))
#         x = self.dropout(self.fc7(x))
#         x = self.dropout(self.fc8(x))
#         x = self.dropout(self.fc9(x))
#         x = self.dropout(self.fc10(x))
#         x = self.dropout(self.fc11(x))
#         x = self.fc12(x)
#         return x

# # 12 Capas Densas con Dropout 0.4
# class MLP(nn.Module):
#     def __init__(self, n_input=110250, n_output=10):
#         super(MLP, self).__init__()
#         self.fc1 = nn.Linear(n_input, 2048)
#         self.fc2 = nn.Linear(2048, 1024)
#         self.fc3 = nn.Linear(1024, 1024)
#         self.fc4 = nn.Linear(1024, 512)
#         self.fc5 = nn.Linear(512, 512)
#         self.fc6 = nn.Linear(512, 256)
#         self.fc7 = nn.Linear(256, 256)
#         self.fc8 = nn.Linear(256, 128)
#         self.fc9 = nn.Linear(128, 128)
#         self.fc10 = nn.Linear(128, 64)
#         self.fc11 = nn.Linear(64, 32)
#         self.fc12 = nn.Linear(32, n_output)
#         self.dropout = nn.Dropout(0.4)

#     def forward(self, x):
#         x = self.dropout(self.fc1(x))
#         x = self.dropout(self.fc2(x))
#         x = self.dropout(self.fc3(x))
#         x = self.dropout(self.fc4(x))
#         x = self.dropout(self.fc5(x))
#         x = self.dropout(self.fc6(x))
#         x = self.dropout(self.fc7(x))
#         x = self.dropout(self.fc8(x))
#         x = self.dropout(self.fc9(x))
#         x = self.dropout(self.fc10(x))
#         x = self.dropout(self.fc11(x))
#         x = self.fc12(x)
#         return x

# #12 capas densas dropout 0.8
# class MLP(nn.Module):
#     def __init__(self, n_input=110250, n_output=10):
#         super(MLP, self).__init__()
#         self.fc1 = nn.Linear(n_input, 2048)
#         self.fc2 = nn.Linear(2048, 1024)
#         self.fc3 = nn.Linear(1024, 1024)
#         self.fc4 = nn.Linear(1024, 512)
#         self.fc5 = nn.Linear(512, 512)
#         self.fc6 = nn.Linear(512, 256)
#         self.fc7 = nn.Linear(256, 256)
#         self.fc8 = nn.Linear(256, 128)
#         self.fc9 = nn.Linear(128, 128)
#         self.fc10 = nn.Linear(128, 64)
#         self.fc11 = nn.Linear(64, 32)
#         self.fc12 = nn.Linear(32, n_output)
#         self.dropout = nn.Dropout(0.8)

#     def forward(self, x):
#         x = self.dropout(self.fc1(x))
#         x = self.dropout(self.fc2(x))
#         x = self.dropout(self.fc3(x))
#         x = self.dropout(self.fc4(x))
#         x = self.dropout(self.fc5(x))
#         x = self.dropout(self.fc6(x))
#         x = self.dropout(self.fc7(x))
#         x = self.dropout(self.fc8(x))
#         x = self.dropout(self.fc9(x))
#         x = self.dropout(self.fc10(x))
#         x = self.dropout(self.fc11(x))
#         x = self.fc12(x)
#         return x


### Arquitecura CNN

In [ ]:
# #2 convoluciones y 1 capa densa
# class CNN(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.conv1 = nn.Conv2d(in_channels = 1,   # 1 canal de entrada.
#                                out_channels = 32, # 32 canales de salida.
#                                kernel_size = 3,   # Kernel 3x3.
#                                stride = 1,        # Pasos de 1.
#                                padding = 1)       # Padding de 1 para mantener el tamaño.
#         self.conv2 = nn.Conv2d(32, 64,
#                                kernel_size = 3, stride = 1, padding = 1)
#         self.fc1 = nn.Linear((64 * (201//4)) * (552//4),
#                              # 201 es el alto del espectograma y 552 ancho
#                              # convolucional se le aplica un maxpool de 2x2.
#                              10)

#     #dividido 2 por max pool o promedio
#     def forward(self, x):
#         x = nn.Identity()(self.conv1(x))
#         x = nn.MaxPool2d(kernel_size = 2, stride = 2)(x)
#         x = nn.Identity()(self.conv2(x))
#         x = nn.MaxPool2d(kernel_size = 2, stride = 2)(x)
#         x = torch.flatten(x, 1)
#         x = nn.Identity()(self.fc1(x))
#         return x

# 2 convoluciones y 5 capas densa
class CNN(nn.Module):
    def __init__(self):
        super().__init__()

        # 🔹 Convoluciones
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32,
                               kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64,
                               kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # 🔹 Tamaño del espectrograma de entrada: [1, 1, 201, 552]
        # Dos maxpools reducen altura y ancho a la mitad dos veces → 201 → 100, 552 → 276
        flattened_size = 64 * (201 // 4) * (552 // 4)  # = 64 * 50 * 138

        # 🔹 Capas densas (sin activación)
        self.fc1 = nn.Linear(flattened_size, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 64)
        self.fc5 = nn.Linear(64, 10)  # 10 géneros

    def forward(self, x):
        x = self.pool(self.conv1(x))
        x = self.pool(self.conv2(x))
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        x = self.fc4(x)
        x = self.fc5(x)
        return x


In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else
                      'mps' if torch.backends.mps.is_available() else
                      'cpu')

# model = MLP(n_input=110250, n_output=len(classes))
# model.to(device)
# print(model)


# def count_parameters(model):
#     return sum(p.numel() for p in model.parameters() if p.requires_grad)


# n = count_parameters(model)
# print("Number of parameters: %s" % n)

#Para imprimir el modelo con CNN
# input_shape = (1, 201, 552)
# model = CNN(input_shape)
model = CNN()
model.to(device)
print(model)

In [ ]:
len(classes)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr= learning_rate)

### Train y Validation para Waveform

In [ ]:
# import gc
# from sklearn.metrics import accuracy_score, confusion_matrix
# import torch.nn.functional as F

# loss_function = nn.CrossEntropyLoss()
# valid_losses = []
# num_epochs = epochs
# torch.cuda.empty_cache()
# gc.collect() #importante para ir liberando memoria ram

# best_valid_accuracy = 0

# for epoch in range(num_epochs):
#     losses = []

#     # Train
#     model.train()
#     for wav, genre_index in train_dl:
#         optimizer.zero_grad()  # Clear gradients
#         wav=wav.to(device)
#         genre_index =torch.as_tensor(genre_index).to(device)

#         # Forward
#         out = model(wav)
#         #M5
#         loss = loss_function(out.squeeze(), genre_index)

#         # Backward
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
#         losses.append(loss.item())

#         del wav #importante para ir liberando memoria ram
#         del genre_index #importante para ir liberando memoria ram
#         del loss #importante para ir liberando memoria ram
#         del out  #importante para ir liberando memoria ram
#         torch.cuda.empty_cache()  #importante para ir liberando memoria ram
#         gc.collect() #importante para ir liberando memoria ram
#         wandb.log({'train_loss': np.mean(losses)}, commit = False)

#     print('Epoch: [%d/%d], Train loss: %.4f' % (epoch+1, num_epochs, np.mean(losses)))

#     # Validation
#     model.eval()
#     y_true = []
#     y_pred = []
#     losses = []
#     correct =0
#     for wav, genre_index in valid_dl:
#         #print(wav, genre, index)
#         wav = wav.to(device)
#         genre_index = genre_index.to(device)

#         out = model(wav)

#         loss = loss_function(out.squeeze() , genre_index)

#         losses.append(loss.item())
#         #M5
#         pred= out.argmax(dim=-1).flatten()
#         # append labels and predictions
#         correct += pred.eq(genre_index).sum().item()
#         y_true.extend(genre_index)
#         y_pred.extend(pred)
#         del wav #importante para ir liberando memoria ram
#         del genre_index #importante para ir liberando memoria ram
#         del loss #importante para ir liberando memoria ram
#         del out  #importante para ir liberando memoria ram
#         torch.cuda.empty_cache()  #importante para ir liberando memoria ram
#         gc.collect() #importante para ir liberando memoria ram

#     valid_accuracy =correct/ len(valid_dl.dataset)
#     valid_loss = np.mean(losses)
#     print('Epoch: [%d/%d], Valid loss: %.4f, Valid accuracy: %.4f' % (epoch+1, num_epochs, valid_loss, valid_accuracy))

#     # Guardar el mejor modelo según la precisión de validación
#     if valid_accuracy > best_valid_accuracy:
#         best_valid_accuracy = valid_accuracy
#         torch.save(model.state_dict(), 'best_model.ckpt')

#     wandb.log({
#         'val_loss': valid_loss,         # Pérdida de validación calculada arriba
#         'val_accuracy': valid_accuracy        # Exactitud de validación calculada arriba
#         })

### Train y validation con Espectrograma

In [ ]:
import gc
from sklearn.metrics import accuracy_score, confusion_matrix
import torch.nn.functional as F

loss_function = nn.CrossEntropyLoss()

valid_losses = []
num_epochs = epochs
delay_epochs = 5

torch.cuda.empty_cache()
gc.collect() #importante para ir liberando memoria ram

best_valid_accuracy = 0

# Definir el tipo de regularización (None, 'L1', o 'L2')
#regularization_type = 'L2'
#lambda_reg = 1e-4
#lambda_reg = 1e-6
#lambda_red = 1e-3
#Esta comentado pues no se usa regularizacion

for epoch in range(num_epochs):
    losses = []

    # Train
    model.train()
    for wav, genre_index in train_dl:
        optimizer.zero_grad()
        wav=wav.to(device)
        genre_index =torch.as_tensor(genre_index).to(device)

        specgram = tt.Spectrogram().to(device)
        specgram_wav = specgram(wav)

        # Forward
        out = model(specgram_wav)

        loss = loss_function(out.squeeze(), genre_index)


         #Regularización L1 o  L2
        #if regularization_type == 'L1':
            #l1_reg = torch.tensor(0., device=device)
            #for param in model.parameters():
                #l1_reg += torch.norm(param, 1)  # Norma L1
            #loss += lambda_reg * l1_reg  # Agregar L1 a la pérdida total
        #elif regularization_type == 'L2':
            #l2_reg = torch.tensor(0., device=device)
            #for param in model.parameters():
                #l2_reg += torch.norm(param, 2)  # Norma L2
            #loss += lambda_reg * l2_reg  # Agregar L2 a la pérdida total

            #Comentado por no se usa regularizacion

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        losses.append(loss.item())

        del wav
        del genre_index
        del loss
        del out
        torch.cuda.empty_cache()
        gc.collect()

    print('Epoch: [%d/%d], Train loss: %.4f' % (epoch+1, num_epochs, np.mean(losses)))

    # Validation
    model.eval()
    y_true = []
    y_pred = []
    losses = []
    correct =0
    for wav, genre_index in valid_dl:
        wav = wav.to(device)
        genre_index = genre_index.to(device)

        specgram = tt.Spectrogram().to(device)
        specgram_wav = specgram(wav)

        out = model(specgram_wav)

        loss = loss_function(out.squeeze(), genre_index)

        losses.append(loss.item())

        pred= out.argmax(dim=-1).flatten()

        correct += pred.eq(genre_index).sum().item()
        y_true.extend(genre_index)
        y_pred.extend(pred)
        del wav
        del genre_index
        del loss
        del out
        torch.cuda.empty_cache()
        gc.collect()

    valid_accuracy =correct/ len(valid_dl.dataset)
    valid_loss = np.mean(losses)
    print('Epoch: [%d/%d], Valid loss: %.4f, Valid accuracy: %.4f' % (epoch+1, num_epochs, valid_loss, valid_accuracy))


    # Guardar el mejor modelo según la precisión de validación
    if valid_accuracy > best_valid_accuracy:
        best_valid_accuracy = valid_accuracy
        torch.save(model.state_dict(), 'best_model.ckpt')

    #Comentado pues no se utiliza en el mejor modelo

    # LR no fijos
    #Decaimiento exponencial
    #if epoch >= delay_epochs: #con if es decaimiento exponencial retrasado
    #scheduler.step()
    #current_lr = scheduler.get_last_lr()[0]
    #print(f"Learning Rate después de la epoch {epoch+1}: {current_lr}")
    #Decaimiento basado en error
    #scheduler.step(valid_loss)
    #current_lr = scheduler.get_last_lr()[0]
    #print(f"Learning Rate después de la epoch {epoch+1}: {current_lr}")



### 5. Evaluación



In [ ]:
test_dl = DataLoader(test_ds,1,shuffle=True, num_workers=0, pin_memory=True)

#### Test para Waveform

In [ ]:
# # Load the best model
# S = torch.load('best_model.ckpt')
# model.load_state_dict(S)
# print('loaded!')

# # Run evaluation
# model.eval()
# y_true = []
# y_pred = []
# correct = 0
# with torch.no_grad():
#     for wav, genre_index in test_dl:
#         wav = wav.to(device)
#         genre_index = genre_index.to(device)

#         out = model(wav)

#         pred= out.argmax(dim=-1).flatten()
#         # append labels and predictions
#         correct += pred.eq(genre_index).sum().item()
#         y_true.extend(genre_index)
#         y_pred.extend(pred)

# test_accuracy = correct/ len(test_dl.dataset)
# print('Epoch: [%d/%d], Test accuracy: %.4f' % (epoch+1, num_epochs, test_accuracy))

# # Registrar la precisión y la pérdida en W&B
# wandb.summary["test_accuracy"]= test_accuracy



### Test para Espectrograma

In [ ]:
# Load the best model
S = torch.load('best_model.ckpt')
model.load_state_dict(S)
print('loaded!')

# Run evaluation
model.eval()
y_true = []
y_pred = []
correct = 0
with torch.no_grad(): #desactiva calculo de gradientes
    for wav, genre_index in test_dl:
        wav = wav.to(device)
        genre_index = genre_index.to(device)

        specgram = tt.Spectrogram().to(device)
        specgram_wav = specgram(wav)

        out = model(specgram_wav)

        pred= out.argmax(dim=-1).flatten()
        # append labels and predictions
        correct += pred.eq(genre_index).sum().item()
        y_true.extend(genre_index)
        y_pred.extend(pred)

test_accuracy = correct/ len(test_dl.dataset)
print('Epoch: [%d/%d], Test accuracy: %.4f' % (epoch+1, num_epochs, test_accuracy))


In [ ]:
# Finish the run and upload any remaining data.
run.finish()

In [ ]:
waveform,label= test_dl.dataset[12]
print("shape of waveform {}, sample rate with {}, label is {} ".format(waveform.size(),samplerate,label))

In [ ]:
import IPython
IPython.display.Audio(waveform, rate=22050)

In [ ]:
wav= torch.unsqueeze(waveform, dim=0)
model.to(device)
wav =wav.to(device)
out = model(specgram_wav) #cambiar a wav para waveform
pred= out.argmax(dim=-1).flatten()
classes[pred], classes[label]

In [ ]:
print(test_accuracy)
print(valid_accuracy)
print(valid_loss)